# ThiNC

THiNC is a deep learning framework that makes composing, configuring and deploying models easy. It provides a flexible yet simple approach to modelling by providing low-level abstractions of the training loop, evaluation loop etc. Moreover, it plays well with major deep learning frameworks like TensorFlow and PyTorch. The functional programming API of THiNC is fairly simple and elegant. It’s light weighted API makes THiNC a good option for quick prototyping and deployment of machine learning models.

# Code Implementation

## Setup

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy sklearn statsmodels tensorflow keras torch torchvision --user -q

In [ ]:
!python -m pip install "thinc==8.0.0rc6.dev0" --user -q
#To use GPU we need to install cupy
!python -m pip install "cupy-cuda101" --user -q

In [ ]:
!python -m pip install "ml_datasets>=0.2.0a0" --user -q


In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import ml_datasets
(train_X, train_Y), (dev_X, dev_Y) = ml_datasets.mnist()
print(f"Training size={len(train_X)}, dev size={len(dev_X)}")

In [ ]:
train_X.shape,train_Y.shape

## Classifier Model

To Build a model all the classes and functions must be imported from thin.api .Let’s build a simple FCNN with dropout to recognize the handwritten digits of MNIST dataset.

In [ ]:
from thinc.api import Model,chain, Relu, Softmax
 
n_hidden1, n_hidden2, n_hidden3 = 64,32,10
dropout = 0.2

model = chain(
    Relu(nO=n_hidden1, dropout=dropout),
    Relu(nO=n_hidden2, dropout=dropout), 
    Relu(nO=n_hidden3, dropout=dropout), 
    Softmax()
)

## Operator Overloading

Operator overloading can be used to build complex models in a concise manner.This FCNN model can be built in a single line by using operator overloading.

In [ ]:
with Model.define_operators({">>": chain}):
    model = Relu(nO=n_hidden1, dropout=dropout) >> Relu(nO=n_hidden2, dropout=dropout) >> Relu(nO=n_hidden3, dropout=dropout) >> Softmax()

## Model Initialization

Input and output shapes, along with all the missing shape information in the model can be inferred automatically by initializing the model with sample inputs.

In [ ]:
# making sure the data is on the right device
train_X = model.ops.asarray(train_X)
train_Y = model.ops.asarray(train_Y)
dev_X = model.ops.asarray(dev_X)
dev_Y = model.ops.asarray(dev_Y)

model.initialize(X=train_X[:5], Y=train_Y[:5])
nI = model.get_dim("nI")
nO = model.get_dim("nO")
print(f"Initialized model with input dimension nI={nI} and output dimension nO={nO}")

## Training the Model

Next, we need to build a training loop. THiNC provides low-level abstractions for batching the data and shuffling it. These help in building custom training loops. The following lines are the backbone of training.

In [ ]:
from tqdm.notebook import tqdm
def train_model(data, model, optimizer, n_iter, batch_size):
    (train_X, train_Y), (dev_X, dev_Y) = data
    indices = model.ops.xp.arange(train_X.shape[0], dtype="i")
    for i in range(n_iter):
        batches = model.ops.multibatch(batch_size, train_X, train_Y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            backprop(Yh - Y)
            model.finish_update(optimizer)
        # Evaluate and print progress
        correct = 0
        total = 0
        for X, Y in model.ops.multibatch(batch_size, dev_X, dev_Y):
            Yh = model.predict(X)
            correct += (Yh.argmax(axis=1) == Y.argmax(axis=1)).sum()
            total += Yh.shape[0]
        score = correct / total
        print(f" {i} {float(score):.3f}")

In [ ]:
from thinc.api import Adam, fix_random_seed
fix_random_seed(0)
optimizer = Adam(0.001)
batch_size = 128
n_iter = 10
train_model(((train_X, train_Y), (dev_X, dev_Y)), model, optimizer, n_iter, batch_size)

## Compatibility with Other Frameworks

THiNC provides awesome wrappers to integrate with Tensorflow, PyTorch and MXNet.These wrappers help a lot when it comes to porting code amongst different frameworks. If some functionality you need is ready in a different framework, then you can use those special layers by wrapping them using the THiNC wrappers for that particular framework.

Example: Let’s implement the same FCNN as above using layers from both TensorFlow and PyTorch.

In [ ]:
from thinc.api import PyTorchWrapper, TensorFlowWrapper, chain, Linear, Adam
dropout=0.2

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
tf_model = Sequential()
tf_model.add(Dense(64, activation="relu", input_shape=(784,)))
tf_model.add(Dropout(dropout))

import torch
import torch.nn
import torch.nn.functional as F
class PyTorchModel(torch.nn.Module):
    def __init__(self, nO, nI, dropout):
        super(PyTorchModel, self).__init__()
        self.dropout1 = torch.nn.Dropout2d(dropout)
        self.fc1 = torch.nn.Linear(nI, nO,)

    def forward(self, x):
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = F.relu(x)
        return x



model = chain(
    TensorFlowWrapper(tf_model),  
    PyTorchWrapper(PyTorchModel( 32, 64, dropout)),
    Relu(nO=10, dropout=dropout), 
    Softmax()
)
model

In [ ]:
model.initialize(X=train_X[:5], Y=train_Y[:5])

In [ ]:
train_model(((train_X, train_Y), (dev_X, dev_Y)), model, optimizer, n_iter, batch_size)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
preds,_=model(train_X,is_train=False)
print('Following image is predicted as {}'.format(np.argmax(preds[0])))
plt.imshow(train_X[0].reshape(28,28),cmap='gray')
plt.show()

## Config System

Configuration is an important aspect of product development. You need models to be manageable. We might often need to expose lots of model components as hyperparameters.

THiNC provides a great config system to do this.We can define trees of hyperparameters using simple json like structures. Then we can resolve these configurations and get the functions to run.Let’s create a config for the classifier model above.

In [ ]:
from thinc.api import Config, registry
CONFIG = """
[hyper_params]
n_hidden1 = 64
n_hidden2 = 32
n_hidden3 = 10
dropout = 0.2


[model]
@layers = "chain.v1"

[model.*.relu1]
@layers = "Relu.v1"
nO = ${hyper_params:n_hidden1}
dropout = ${hyper_params:dropout}

[model.*.relu2]
@layers = "Relu.v1"
nO = ${hyper_params:n_hidden2}
dropout = ${hyper_params:dropout}

[model.*.relu3]
@layers = "Relu.v1"
nO = ${hyper_params:n_hidden3}
dropout = ${hyper_params:dropout}

[model.*.softmax]
@layers = "Softmax.v1"

[optimizer]
@optimizers = "Adam.v1"


[optimizer.learn_rate]
@schedules = "warmup_linear.v1"
initial_rate = 2e-5
warmup_steps = 1000
total_steps = 10000

[training]
n_iter = 10
batch_size = 128
"""

config = Config().from_str(CONFIG)
config

In [ ]:
loaded_config = registry.resolve(config)
loaded_config

In [ ]:
loaded_config = registry.resolve(config)
model = loaded_config["model"]
optimizer = loaded_config["optimizer"]
n_iter = loaded_config["training"]["n_iter"]
batch_size = loaded_config["training"]["batch_size"]

model.initialize(X=train_X[:5], Y=train_Y[:5])
train_model(((train_X, train_Y), (dev_X, dev_Y)), model, optimizer, n_iter, batch_size)

In [ ]:
y_true=np.argmax(train_Y,axis=1)
y_pred=np.argmax(preds,axis=1)

In [ ]:
def accuracy(y_true,y_pred):
  assert len(y_true)==len(y_pred),"Lengths of labels mismatched"
  assert len(y_pred)!=0,"Predictions are empty"
  return sum([1 if i==j else 0 for i,j in zip(y_true,y_pred)])/len(y_pred)
#Accuracy is on the scale of 0 to 1.
accuracy(y_true,y_pred)


In [ ]:
from sklearn.metrics import precision_score
precision_score(y_true,y_pred,average=None)

In [ ]:
import numpy as np
def multiclass_precision(y_true,y_pred):
  assert len(y_true)==len(y_pred),"Lengths of labels mismatched"
  assert len(y_pred)!=0,"Predictions are empty"
  precisions=[]
  for class_ in sorted(np.unique(y_true)):
    indices = np.array(y_pred==class_).nonzero()[0]
    try:
      precisions.append(sum(y_true[indices]==class_)/len(indices))
    except ZeroDivisionError as e:
      precisions.append(0)
  return precisions
#Accuracy is on the scale of 0 to 1.
def precision_score(y_true,y_pred,aggregator=None):
  try:
    return multiclass_precision(y_true,y_pred)
  except AssertionError as a:
    print(a)
    raise KeyError
  except TypeError:
    return accuracy(*list(zip(*list(filter(lambda x:x[1],zip(y_true,y_pred)))))) if sum(y_pred)!=0 else 0

# Precision is on the scal eof 0 to 1
precision_score(y_true,y_pred)

In [ ]:
def multiclass_recall(y_true,y_pred):
  assert len(y_true)==len(y_pred),"Lengths of labels mismatched"
  assert len(y_pred)!=0,"Predictions are empty"
  recalls=[]
  for class_ in sorted(np.unique(y_true)):
    indices = np.array(y_true == class_).nonzero()[0]
    try:
      recalls.append(sum(y_pred[indices]==class_)/len(indices))
    except ZeroDivisionError as e:
      recalls.append(0)
  return recalls
#Accuracy is on the scale of 0 to 1.
def recall_score(y_true,y_pred,aggregator=None):
  try:
    return multiclass_recall(y_true,y_pred)
  except AssertionError as a:
    print(a)
    raise KeyError
  except TypeError:
    return accuracy(*list(zip(*list(filter(lambda x:x[0],zip(y_true,y_pred)))))) if sum(y_pred)!=0 else 0

#Recall is on the scale of 0 to 1
recall_score(y_true,y_pred)

In [ ]:
import operator
def f1_score(y_true,y_pred):
  try:
    prcsn,rcl = precision_score(y_true,y_pred),recall_score(y_true,y_pred)
    return (2*np.multiply(prcsn,rcl))/np.add(prcsn,rcl)
  except ZeroDivisionError:
    return 0
#f1_score is on the scale of 0 to 1
f1_score(y_true,y_pred)

In [ ]:
2*np.multiply([2,3],[3,4,])

In [ ]:
np.add([2,3],[3,4,])